In [ ]:
import pandas as pd
import os
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import lightgbm as lgb
import streamlit as st
import shap
import seaborn as sns
import numpy as np
from unidecode import unidecode
from tqdm import tqdm


# Read data

## Set current directory

In [ ]:
# Set current directory
os.chdir("/Users/othmanbensouda/Desktop/Capstone project/Data/main_data")

## Main datasets

In [ ]:
%%time

academics = pd.read_csv("Academics.csv")

classes = pd.read_excel("SchoolsClasses.xlsx",sheet_name  = 0) 


In [ ]:
%%time

schools =  pd.read_excel("SchoolsClasses.xlsx",sheet_name  = 1)

staff = pd.read_csv("EnseignantNiveauxClasses.csv")

students = pd.read_csv("Demographics.csv")

## Nomenclature

In [ ]:
%%time

years_nomenclature = pd.read_excel("final_data_dictionary.xlsx",sheet_name = 7, skiprows = 1)

subjects_nomenclature = pd.read_excel("final_data_dictionary.xlsx",sheet_name = 8, skiprows= 1)

levels_nomenclature = pd.read_excel("final_data_dictionary.xlsx",sheet_name = 9,skiprows= 1)

handicap_nomenclature = pd.read_excel("final_data_dictionary.xlsx",sheet_name = 10,skiprows = 1)

student_situation_nomenclature = pd.read_excel("final_data_dictionary.xlsx",sheet_name = 11,skiprows= 1)

results_nomenclature = pd.read_excel("final_data_dictionary.xlsx",sheet_name = 12,skiprows= 1)

scholarship_nomenclature = pd.read_excel("final_data_dictionary.xlsx",sheet_name = 13)

position_nomenclature = pd.read_excel("final_data_dictionary.xlsx",sheet_name = 14,skiprows = 1)

# Data cleaning

## Schools

In [ ]:
pd.set_option('display.max_columns', None)


In [ ]:
# Remove unecessary columns
columns_to_drop = ["CD_etabr","NetabFr","Nbre_Satellites","Superf_etab","Superf_Cour","Cantine_Capacite","Superf_EspVert","Superf_Exten","Cantine_DO","cd_Ass","Cantine_DF","Couvert_place","Internat_Capacite","Internat_DO","Internat_DF","CD_ETAB.1","ll_reg","cd_prov","NombreSurvGenExt","AdresseL","CD_NETAB","CD_Financ", "DC_ETAB", "DF_ETAB", "DistSatEcoM", "DistUniteCol", "DistColLyc", "Eau_Source",'DO_ETAB', "Elec_Source", "Ass_Source", "FP_Bac"]
schools.drop(columns_to_drop, axis=1, inplace=True)

In [ ]:
schools

## Classes

In [ ]:
## BE CAREFUL, THERE ARE CLASSES THAT CAN HAVE LOTS OF DIFFERENT LEVELS AND APPEAR MANY TIMES IN THE DATASET, 
## EVEN THOUGH THEY ARE NOT DUPLICATES PER SE
classes.duplicated().sum()

In [ ]:
def add_occurrences_column(dataframe):
    grouped = dataframe.groupby('id_classe')
    dataframe['nombre_de_niveaux'] = grouped['id_classe'].transform('count')
    return dataframe

In [ ]:
classes = add_occurrences_column(classes)

In [ ]:
# Create parity column
classes['parite'] = classes['nbr_filles'] / classes['nbr_eleves']


In [ ]:
classes

# Students

In [ ]:
# Merge with handicap nomenclature
students = students.merge(handicap_nomenclature[["id_handicap","handicapFr"]], how = "left", left_on = "id_handicap", right_on = "id_handicap" )
# Change NaN to no handicap
students["handicapFr"].fillna("no handicap", inplace=True)
# Drop id_handicap column and adress
students.drop(["id_handicap","Adress"], axis=1, inplace=True)


In [ ]:
# Change nationality to either Moroccan, other, or NaN
students['nationalite'] = np.where((students['nationalite'] != 'MAR') & (~students['nationalite'].isnull()), 'other', students['nationalite'])
# Replace Nan by not specified
students['nationalite'].fillna('not specified', inplace=True)
# Remove accents and convert to lowercase
students['Lieu_naissance_fr'] = students['Lieu_naissance_fr'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
students['Lieu_naissance_fr'] = students['Lieu_naissance_fr'].str.lower()


In [ ]:
# Fill NA for profession_mere and profession_pere
students['profession_pere'] = students['profession_pere'].fillna('non_specifie').astype(str)
students['profession_mere'] = students['profession_pere'].fillna('non_specifie').astype(str)



In [ ]:
## HERE, INSTEAD OF HARDCODING, USE CHATGPT API. THIS HAS BEEN DONE WITH CHATGPT

jobs_father = {
    'Laborer': [
        'عامل',
        'مياوم',
        'عامل فلاحي',
        'JOURNALIER',
        'OUVRIER',
        'عامل يومي',
        'عامل بناء',
        'fellah',
        'عامل فلا حي',
        'ouvrier',
        'مزارع',
        'مياو'
    ],
    'Trader': [
        'تاجر',
        'بائع متجول',
        'مساعد تاجر',
        'بائع',
        'تاجر متجول',
        'بائع السمك',
        'Commerçant',
        'بائع خضر',
        'بائع سمك'
    ],
    'Driver': [
        'سائق',
        'سائق طاكسي',
        'سائق شاحنة',
        'سائق سيارة أجرة'
    ],
    'Retired': [
        'متقاعد',
        'جندي متقاعد',
        'عسكري متقاعد',
        'متوفى'
    ],
    'Employee': [
        'موظف',
        'مستخدم',
        'مساعد تقني',
        'مستخدم بشركة',
        'مدير شركة',
        'مدير',
        'مسؤول تجاري',
        'مساعد صيدلي',
        'مدير تجاري',
        'مستخدم بشركة',
        'مدير مدرسة',
        'EMPLOYE'
    ],
    'Craftsman': [
        'خياط',
        'نجار',
        'صباغ',
        'ميكانيكي',
        'صانع تقليدي',
        'زلايجي',
        'لحام',
        'حلاق',
        'خراز',
        'صانع أحذية',
        'فندقي',
        'صانع',
        'فخاري',
        'مكانيكي',
        'MACON',
        'TAILLEUR',
        'صانع أسنان'
    ],
    'Teacher': [
        'أستاذ',
        'أستاذ جامعي',
        'استاذ',
        'أستاذ التعليم الابتدائي',
        'استاذ التعليم الابتدائي'
    ],
    'Security': [
        'عون سلطة',
        'حارس',
        'حارس ليلي',
        'حارس عمارة',
        'حارس امن',
        'حارس عام',
        'حارس أمن'
    ],'Military': [
        'جندي',
        'عسكري',
        'جندي متقاعد',
        'عسكري متقاعد',
        'SOLDAT',
        'القوات المساعدة'
    ],
    'Worker': [
        'بناء',
        'جباص',
        'دركي',
        'مياوم',
        'دراز',
        'عاطل',
        'اجير',
        'موطف',
        'موسمي',
        'مهاجر',
        'عمل حر',
        'آخر',
        'بدون',
        'مياوم',
        'عامل يومي',
        'لاشيء',
        'أعمال حرة',
        'fonctionnaire',
        'موظف',
        'RETRAITE',
        'PROFESSEUR',
        'FONCTIONNAIRE',
        'EMPLOYE',
        'commercant',
        '  عامل',
        'commerçant',
        'EMPLOYE',
        'commercant',
        'métier'
    ],
    'Officer': [
        'ضابط',
        'ضابط صف',
        'ضابط سامي',
        'ضابط شرطة',
        'مسؤول تجاري',
        'ضابط سامي'
    ],
    'Technician': [
        'مهندس',
        'تقني',
        'ميكانيكي',
        'تقني فلاحي',
        'مهندس دولة',
        'مساعد تقني',
        'تقني متخصص',
        'إطار بنكي',
        'تقني',
        'Mécanicien',
        'MACON',
        'التجارة'
    ],
    'Healthcare Professional': [
        'ممرض',
        'طبيب',
        'طباخ',
        'صيدلي',
        'طبيب',
        'ممرض',
        'أطباء',
        'طباخ',
        'أخصائي تقويم',
        'نفساني',
        'ممرضة',
        'صيدلاني',
        'دكتور',
        'طبيب',
        'صيدلي',
        'طبيب أطفال',
        'مسعف',
        'مستشفى'
    ],
    'Retired': [
        'متقاعد',
        'جندي متقاعد',
        'عسكري متقاعد',
        'RETRAITE'
    ],
    'Agriculturist': [
        'فلاح',
        'عامل فلاحي',
        'fellah',
        'FELLAH',
        'فلاح مياوم',
        'مياوم فلاحي',
        'فلاح\t'
    ],    'Artisan': [
        'مصور',
        'موسيقي',
        'مقدم',
        'ميكانيك',
        'كباص',
        'صاحب مقهى',
        'صانع',
        'فخاري',
        'صانع أحذية',
        'مصمم',
        'صانع مجوهرات',
        'فنان',
        'عامل خزف',
        'فنان تشكيلي',
        'خياط',
        'نجار',
        'صباغ',
        'ميكانيكي',
        'حلاق',
        'خراز',
        'صانع أثاث',
        'فنان تصوير',
        'مطرب',
        'فنانة',
        'مصفف شعر',
        'خياطة',
        'فنان بصري',
        'موسيقار',
        'صانع زجاج',
        'مصمم أزياء',
        'خزاف',
        'فنان رقص',
        'عازف',
        'صانع ساعات',
        'فنان تاتو',
        'صانع نحاس',
        'عازفة',
        'فنان منحوتات',
        'صانع فخار',
        'فنان جرافيك',
        'عازفة بيانو',
        'صانع سجاد',
        'صانع دمى',
        'فنان تشكيلي',
        'عازف غيتار',
        'صانع تحف',
        'صانع تحف',
        'فنانة رقص',
        'معلم رسم',
        'عازف كمان',
        'صانع أعمال خشبية',
        'فنانة جرافيك',
        'عازفة كمان',
        'صانع مفروشات',
        'صانع ألعاب',
        'فنان بالونات',
        'صانع مجوهرات',
        'عازفة بيانو',
        'صانع زجاج',
        'فنان تصوير',
        'صانع ساعات',
        'صانع تحف',
        'فنان جرافيك',
        'صانع سجاد',
        'صانع دمى',
        'فنان تشكيلي',
        'صانع أعمال خشبية',
        'عازف كمان',
        'صانع مفروشات',
        'صانع ألعاب',
        'فنان بالونات',
        'فنان ماكياج',
        'صانع مجوهرات',
        'صانع نحاس',
        'فنان تشكيلي',
        'عازفة بيانو',
        'صانع']}


In [ ]:
jobs_mother = {
    'Homemaker/Unemployed': ['ربة بيت', 'ربت بيت', 'ربة  بيت', 'ربــة بــيــت', 'ربــة  بـيــت', 'ربة بية', 'ربة بيث', 'ربة منزل', 'ربت منزل', 'ر بة بيت', 'ربة ربيت', 'ربة بييت', 'ربة', 'ربت البيت', 'ربية بيت', 'ربة   بيت', 'ربة البيت', 'ربةبيت', 'ربــة بـيــت', 'ربة بييت', 'ربة ربيت', 'FEMME AU FOYER', 'FEMME DE FOYER', 'Femme de foyer', 'femme au foyer', 'femme de foyer', 'اشغال البيت', 'اشغال المنزل', 'البيت', 'بدون', 'SANS', 'sans', 'Sans', 'SAN', 'لاشيء', 'لا شيء', 'لاشئ', 'لاشيئ', 'لا شىء', 'لا شئ', 'لاشىء', 'NEANT', 'RIEN', 'بذون', 'بدون مهنة', 'بدزن', '-', 'ب', 'دون', 'يدون'],
    'Worker': ['عاملة', 'مياومة', 'منظفة', 'عامل', 'مياوم', 'اعمال حرة', 'فلاح', 'فلاحة', 'حلاقة', 'خادمة', 'نادلة', 'عاملة نظافة', 'عاملة بشركة', 'سائق', 'صانعة تقليدية', 'خياط', 'خيا','طة', 'طرازة', 'مولدة', 'بناء', 'مقاولة', 'حلوانية', 'مؤطرة'],
    'Educator': ['أستاذة', 'استاذة', 'معلمة', 'أستاذ', 'مدرسة', 'أستاذة التعليم الابتدائي', 'استاذة التعليم الابتدائي', 'أستاذة جامعية', 'ENSEIGNANTE', 'معلم(ة)'],
    'Employee': ['موظفة', 'مستخدمة', 'موظف', 'متصرفة', 'FONCTIONNAIRE'],
    'Medical profession': ['ممرضة', 'طبيبة', 'صيدلانية', 'قابلة', 'مساعدة صيدلي'],
    'Business': ['تاجرة', 'تاجر', 'مديرة تربوية', 'مديرة', 'مسيرة شركة', 'إطار بنكي'],
    'Farmer': ['فلاح', 'فلاحة'],
    'Writer': ['كاتبة'],
    'Teacher': ['أستادة', 'أستاذة', 'استاذة'],
    'Retired': ['متقاعدة'],
    'Student': ['طالبة'],
    'Manager': ['مديرة', 'مديرة تربوية', 'مسيرة شركة'],
    'Driver': ['سائق'],
    'Other': ['مربية', 'متوفية', 'متوفاة', 'مطلقة', 'مرافقة', 'زبة بيت', 'عسكرية', 'مولدة', 'عاملة بشركة', 'مقاولة', 'VF? FDJ', 'بة بيت', 'ربة', 'زبة بيت', 'مساعدة اجتماعية', 'COUTURIERE', 'enfoye', 'F£.K', 'مصممة أزياء', 'مؤطرة']
}


In [ ]:
# Replace the words with categories in the students DataFrame
for category, words in jobs_mother.items():
    students['profession_mere'] = students['profession_mere'].replace(words, category)

for category, words in jobs_father.items():
    students['profession_pere'] = students['profession_pere'].replace(words, category)


# Academics

In [ ]:
# Drop duplicates
academics = academics.drop_duplicates()

In [ ]:
academics.head()

In [ ]:
# Select only from 2015-2016 to 2018-2019 : Remove covid years
academics = academics[(academics['id_annee'] >= 8) & (academics['id_annee'] <= 11)]


In [ ]:
# Merge with levels nomenclature to get corresponding nefstat
academics = academics.merge(levels_nomenclature[["nefstat","CD_CYCLE","id_typeEnseignement","libformatFr","Suffix"]],how = "left", left_on = "nefstat", right_on = "nefstat" )

In [ ]:
# Remove higher education (CD cycle starting with 4) and Preschool (CD cycle starting with 00)
academics = academics[~academics['CD_CYCLE'].astype(str).str.startswith('4')]
academics = academics[academics['CD_CYCLE'].astype(str) != '00']

In [ ]:
# Fix problem with absences
def fix_absences(dataset):
    # Group by student, year, and session
    grouped = dataset.groupby(['id_eleve', 'id_annee', 'id_session'])
    
    # Apply the desired calculations to each group
    dataset['NbrJourAbsenceAutorise'] = grouped['NbrJourAbsenceAutorise'].transform('max')
    dataset['NbrUniteAbsenceAutorise'] = grouped['NbrUniteAbsenceAutorise'].transform('sum')
    dataset['NbrJourAbsenceNonAutorise'] = grouped['NbrJourAbsenceNonAutorise'].transform('sum')
    dataset['NbrUniteAbsenceNonAutorise'] = grouped['NbrUniteAbsenceNonAutorise'].transform('sum')
    
    dataset.drop_duplicates(inplace=True)
    
    return dataset

In [ ]:
fixed_academics = fix_absences(academics)

In [ ]:
fixed_academics

In [ ]:
fixed_academics = fixed_academics.drop(['nefstat_orientation','MoyenneExam'],axis = 1)

In [ ]:
fixed_academics['id_typeBourse'] = fixed_academics['id_typeBourse'].fillna(0)


# Final Dataset

In [ ]:
# Compose the dataset of high school students present in year 10
school_students_id = fixed_academics[(fixed_academics["CD_CYCLE"].isin(['2A','3A'])) & (fixed_academics["id_annee"] == 10) & (fixed_academics["id_session"] == 2) & (~fixed_academics["MoynneCC"].isnull())]["id_eleve"]
school_dataset_academics = fixed_academics[fixed_academics['id_eleve'].isin(school_students_id)]


In [ ]:
school_dataset_academics

In [ ]:
# Check whether there remains students with multiple rows for the same id_session
def find_students_with_multiple_rows(dataframe):
    # Group the dataframe by 'id_eleve' and 'id_annee', and count the number of rows for each group
    grouped_dataframe = dataframe.groupby(['id_eleve', 'id_annee','id_session']).size().reset_index(name='count')

    # Filter the groups with more than 3 rows
    multiple_rows = grouped_dataframe[grouped_dataframe['count'] > 3]

    # Extract the unique student IDs from the filtered groups
    student_ids = set(multiple_rows['id_eleve'].unique())

    return student_ids


In [ ]:
find_students_with_multiple_rows(school_dataset_academics)

# Merge datasets

## Academics with Classes

In [ ]:
# First, I have to make some modifications to the classes dataframe 
# Remove nefstat
classes_modif = classes.drop(['nefstat','id_annee','cd_etab'], axis=1)
classes_modif = classes_modif.drop_duplicates()


In [ ]:
classes_modif

In [ ]:
merged_dataset = pd.merge(school_dataset_academics, classes_modif, how="left", left_on="id_classe", right_on = "id_classe")


In [ ]:
merged_dataset

# Academics & Classes with Students

In [ ]:
merged_dataset = pd.merge(merged_dataset, students, how = "left", on = "id_eleve")


In [ ]:
merged_dataset

# Academics & Classes & Students with Schools

In [ ]:
merged_dataset = pd.merge(merged_dataset, schools, how = "left", left_on = "cd_etab", right_on = 'CD_ETAB')


In [ ]:
merged_dataset

# Create Label

In [ ]:
## REMOVE DROPOUTS OF YEAR 10

# Filter the dataframe based on conditions
remove = merged_dataset[(merged_dataset["MoyenneGen"].isnull()) & (merged_dataset["id_annee"] == 10)]
students_to_remove = remove["id_eleve"].unique()

# Remove students from the merged_dataset
merged_dataset = merged_dataset[~merged_dataset["id_eleve"].isin(students_to_remove)]

In [ ]:
merged_dataset

In [ ]:
# Find ID of dropouts in year 11
def find_dropouts(dataframe):
    # Check for students present in year 10 but not in year 11
    year10_students = set(dataframe[(dataframe["id_annee"] == 10) &
                                    (~dataframe["CD_CYCLE"].str.startswith("2BAC","21BAC")) &
                                    (dataframe["id_resultat"] != 1)]["id_eleve"])

    year11_students = set(dataframe[dataframe["id_annee"] == 11]["id_eleve"])
    dropout_students = year10_students - year11_students

    # Check for students present in year 11 with null MoyenneGen
    year11_null_moyenne = set(dataframe[(dataframe["id_annee"] == 11) & (dataframe["MoyenneGen"].isnull())]["id_eleve"])
    dropout_students.update(year11_null_moyenne)

    return dropout_students


In [ ]:
# Group by id_classe and id_annee, and calculate the rank percentile
merged_dataset['RankPercentile_semester'] = merged_dataset.groupby(['id_classe', 'id_annee','id_session'])['MoynneCC'].rank(pct=True)
merged_dataset['RankPercentile_year'] = merged_dataset.groupby(['id_classe', 'id_annee'])['MoyenneGen'].rank(pct=True)


In [ ]:
# Find dropouts
list_dropouts = find_dropouts(merged_dataset)

In [ ]:
len(list_dropouts)

In [ ]:
merged_dataset["id_eleve"].nunique()

In [ ]:
# Remove year 11
merged_dataset = merged_dataset[merged_dataset['id_annee'] != 11]


In [ ]:
merged_dataset

In [ ]:
# Drop id_situation
merged_dataset = merged_dataset.drop("id_situation", axis = 1)


In [ ]:
merged_dataset

In [ ]:
# Check column types and modify them accordinly
pd.set_option('display.max_rows', None)
print(merged_dataset.dtypes)


In [ ]:


bools = ['INDHcom','INDHquart','istayssir','MCaRtable','ClubEnv',
    'PrescolaireModerne',
    'PrimaireGeneral',
    'PrimaireOriginel',
    'CollegialGeneral',
    'CollegialOriginel',
    'QualifiantGeneral',
    'QualifiantOriginel',
    'QualifiantTechnique',
    'CPGE',
    'BTS',
    'ConsGestEtab',
    'ConsPedag',
    'ConsEnseig',
    'ConsClas',
    'AssParents',
    'AssSportive',
    'CoopScolaire',
    'Partenariat',
    'ClubSante',
    'CentreAppui',
    'AssEcolReussite',
    'ProgrammeTissir','Exist_Internet',
    'FP',
    'ENF',
    'Internat',
    'Restauration',
    'Cantine',
    'Clo_Dur',
    'Clo_partie',
    'Clo_grillage',
    'Clo_bois',
    'Clo_autre',
    'Clo_sans',
    'Const_Dur',
    'Const_Prefab',
    'Const_Pise',
    'Const_Autre'
]

for category in bools:
    merged_dataset[category] = merged_dataset[category].astype(bool)





In [ ]:
objects = ['id_eleve',
    'id_annee',
    'id_resultat',
    'id_session',
    'id_typeBourse',
    'id_genre',
    'CD_REG','id_typeEnseignement']

for column in objects:
    merged_dataset[column] = merged_dataset[column].astype(object)

In [ ]:
# For each column in my dataset, if the type of the column is int or float, replace nan values by median. if the type is not int nor float, replace nan values by mode

for col in merged_dataset.columns:
    if merged_dataset[col].dtype in ['int64', 'float64']:
        merged_dataset[col].fillna(merged_dataset[col].median(), inplace=True)
    else:
        merged_dataset[col].fillna(merged_dataset[col].mode()[0], inplace=True)


In [ ]:
# Filter rows where CD_CYCLE is '10'
merged_dataset.loc[merged_dataset["CD_CYCLE"] == "10", ["MoynneCC", "MoyenneGen"]] *= 2

In [ ]:
# Generate a 'period' column to distinguish the lags
merged_dataset['period'] = merged_dataset['id_annee'].astype(str) + '_sem_' + merged_dataset['id_session'].astype(str)

# Drop 'year' and 'semester' as we no longer need them.
merged_dataset.drop(['id_annee', 'id_session'], axis=1, inplace=True)

# Pivot the table to create unique students per row and periods as columns
pivot = pd.pivot_table(merged_dataset, index='id_eleve', columns='period')

# Flatten the multi-index columns and rename
pivot.columns = ['_'.join(col) + '_lag' for col in pivot.columns]

# Reset the index to convert student_id back to a column
pivot = pivot.reset_index()


In [ ]:
pivot

In [ ]:
# Sort the DataFrame index
pivot.sort_index(inplace=True)

# Then apply forward fill
pivot = pivot.ffill()



In [ ]:
pivot

In [ ]:
# Create a new column 'is_dropout' and initialize it with 0
pivot['is_dropout'] = 0

# Set 'is_dropout' to 1 for IDs present in 'list_dropouts'
pivot.loc[pivot['id_eleve'].isin(list_dropouts), 'is_dropout'] = 1

In [ ]:
pivot

# LightGBM

In [69]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Let's assume 'pivot' is your final DataFrame and 'dropout' is your target variable

# Split data into features (X) and target (y)
X = pivot.drop('is_dropout', axis=1)
y = pivot['is_dropout']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

# Calculate the class imbalance ratio
class_counts = y_train.value_counts()
imbalance_ratio = class_counts[0] / class_counts[1]

# Define hyperparameters manually
params = {
    'scale_pos_weight': 8
}

# Train the LightGBM model using specified parameters
model = LGBMClassifier(**params)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Print the evaluation metrics
print("Accuracy: ", accuracy)
print("Confusion Matrix: \n", confusion)
print("Classification Report: \n", report)



Accuracy:  0.8705879537677379
Confusion Matrix: 
 [[66194  9284]
 [ 1532  6568]]
Classification Report: 
               precision    recall  f1-score   support

           0       0.98      0.88      0.92     75478
           1       0.41      0.81      0.55      8100

    accuracy                           0.87     83578
   macro avg       0.70      0.84      0.74     83578
weighted avg       0.92      0.87      0.89     83578



# XGBoost (SMOTE)

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Let's assume 'pivot' is your final DataFrame and 'is_dropout' is your target variable

# Split data into features (X) and target (y)
X = pivot.drop('is_dropout', axis=1)
y = pivot['is_dropout']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=45)

# Define the parameter grid for grid search
param_grid = {
    'learning_rate': [0.01],
    'max_depth': [20],
    'min_child_weight': [1],
    'subsample': [0.5],
    'colsample_bytree': [0.5],
    'gamma': [0.3]
}

# Initialize the XGBoost classifier
model = XGBClassifier(eval_metric='logloss')

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='recall', cv=2)
grid_search.fit(X_train, y_train)

# Get the best parameters and evaluate the model on the test set
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Predict on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Print the best parameters and evaluation metrics
print("Best Parameters: ", best_params)
print("Accuracy: ", accuracy)
print("Confusion Matrix: \n", confusion)
print("Classification Report: \n", report)




# XGBOOST (Scale pos weights)

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Let's assume 'pivot' is your final DataFrame and 'is_dropout' is your target variable

# Split data into features (X) and target (y)
X = pivot.drop('is_dropout', axis=1)
y = pivot['is_dropout']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

# Calculate the class imbalance ratio
class_counts = y_train.value_counts()
imbalance_ratio = class_counts[0] / class_counts[1]

# Define hyperparameters manually
params = {
    'scale_pos_weight': imbalance_ratio
}

# Train the XGBoost model using specified parameters
model = XGBClassifier(**params)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Print the evaluation metrics
print("Accuracy: ", accuracy)
print("Confusion Matrix: \n", confusion)
print("Classification Report: \n", report)



# Catboost

In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Let's assume 'pivot' is your final DataFrame and 'dropout' is your target variable

# Split data into features (X) and target (y)
X = pivot.drop('is_dropout', axis=1)
y = pivot['is_dropout']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=45)

# Calculate the class imbalance ratio
class_counts = y_train.value_counts()
imbalance_ratio = class_counts[0] / class_counts[1]

# Train the CatBoost model
model = CatBoostClassifier(scale_pos_weight=imbalance_ratio)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Confusion Matrix: \n", confusion_matrix(y_test, y_pred))
print("Classification Report: \n", classification_report(y_test, y_pred))


# Random Forest (SMOTE)

In [75]:
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, fbeta_score

# Assuming 'pivot' is your final DataFrame and 'is_dropout' is your target variable

# Split data into features (X) and target (y)
X = pivot.drop('is_dropout', axis=1)
y = pivot['is_dropout']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=45)

# Apply standard scaling
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

# Apply SMOTE on the training data
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train_scaled, y_train)

# Now you can fit the model on the training data
model = RandomForestClassifier(random_state=45, class_weight="balanced")
model.fit(X_train_res, y_train_res)

# Predict on the validation set with adjusted threshold
y_val_pred_proba = model.predict_proba(X_test_scaled)
y_val_pred = (y_val_pred_proba[:, 1] >= 0.4).astype(int)

# Evaluate the model on the validation set
accuracy_val = accuracy_score(y_test, y_val_pred)
confusion_val = confusion_matrix(y_test, y_val_pred)
report_val = classification_report(y_test, y_val_pred)

# Calculate the F2 score on the validation set
f2_val = fbeta_score(y_test, y_val_pred, beta=2)

# Print the evaluation metrics for the validation set
print("Validation Accuracy: ", accuracy_val)
print("Validation Confusion Matrix: \n", confusion_val)
print("Validation Classification Report: \n", report_val)
print("Validation F2 Score: ", f2_val)

# Predict on the test set with adjusted threshold
y_test_pred_proba = model.predict_proba(X_test_scaled)
y_test_pred = (y_test_pred_proba[:, 1] >= 0.4).astype(int)

# Evaluate the model on the test set
accuracy_test = accuracy_score(y_test, y_test_pred)
confusion_test = confusion_matrix(y_test, y_test_pred)
report_test = classification_report(y_test, y_test_pred)

# Calculate the F2 score on the test set
f2_test = fbeta_score(y_test, y_test_pred, beta=2)

# Print the evaluation metrics for the test set
print("Test Accuracy: ", accuracy_test)
print("Test Confusion Matrix: \n", confusion_test)
print("Test Classification Report: \n", report_test)
print("Test F2 Score: ", f2_test)


Validation Accuracy:  0.903387031996073
Validation Confusion Matrix: 
 [[55817  4653]
 [ 1842  4915]]
Validation Classification Report: 
               precision    recall  f1-score   support

           0       0.97      0.92      0.95     60470
           1       0.51      0.73      0.60      6757

    accuracy                           0.90     67227
   macro avg       0.74      0.83      0.77     67227
weighted avg       0.92      0.90      0.91     67227

Validation F2 Score:  0.671521477757132
Test Accuracy:  0.903387031996073
Test Confusion Matrix: 
 [[55817  4653]
 [ 1842  4915]]
Test Classification Report: 
               precision    recall  f1-score   support

           0       0.97      0.92      0.95     60470
           1       0.51      0.73      0.60      6757

    accuracy                           0.90     67227
   macro avg       0.74      0.83      0.77     67227
weighted avg       0.92      0.90      0.91     67227

Test F2 Score:  0.671521477757132


# Random Forest (Class weights)

In [76]:
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, fbeta_score

# Assuming 'pivot' is your final DataFrame and 'is_dropout' is your target variable

# Split data into features (X) and target (y)
X = pivot.drop('is_dropout', axis=1)
y = pivot['is_dropout']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=45)

# Apply standard scaling
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

# Apply SMOTE on the training data
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train_scaled, y_train)

# Now you can fit the model on the training data
class_counts = y_train_res.value_counts()
imbalance_ratio = class_counts[0] / class_counts[1]
class_weights = {0: 1, 1: imbalance_ratio}
model = RandomForestClassifier(random_state=45, class_weight=class_weights)
model.fit(X_train_res, y_train_res)

# Predict on the validation set with adjusted threshold
y_val_pred_proba = model.predict_proba(X_test_scaled)
y_val_pred = (y_val_pred_proba[:, 1] >= 0.4).astype(int)

# Evaluate the model on the validation set
accuracy_val = accuracy_score(y_test, y_val_pred)
confusion_val = confusion_matrix(y_test, y_val_pred)
report_val = classification_report(y_test, y_val_pred)

# Calculate the F2 score on the validation set
f2_val = fbeta_score(y_test, y_val_pred, beta=2)

# Print the evaluation metrics for the validation set
print("Validation Accuracy: ", accuracy_val)
print("Validation Confusion Matrix: \n", confusion_val)
print("Validation Classification Report: \n", report_val)
print("Validation F2 Score: ", f2_val)

# Predict on the test set with adjusted threshold
y_test_pred_proba = model.predict_proba(X_test_scaled)
y_test_pred = (y_test_pred_proba[:, 1] >= 0.4).astype(int)

# Evaluate the model on the test set
accuracy_test = accuracy_score(y_test, y_test_pred)
confusion_test = confusion_matrix(y_test, y_test_pred)
report_test = classification_report(y_test, y_test_pred)

# Calculate the F2 score on the test set
f2_test = fbeta_score(y_test, y_test_pred, beta=2)

# Print the evaluation metrics for the test set
print("Test Accuracy: ", accuracy_test)
print("Test Confusion Matrix: \n", confusion_test)
print("Test Classification Report: \n", report_test)
print("Test F2 Score: ", f2_test)


Validation Accuracy:  0.903387031996073
Validation Confusion Matrix: 
 [[55817  4653]
 [ 1842  4915]]
Validation Classification Report: 
               precision    recall  f1-score   support

           0       0.97      0.92      0.95     60470
           1       0.51      0.73      0.60      6757

    accuracy                           0.90     67227
   macro avg       0.74      0.83      0.77     67227
weighted avg       0.92      0.90      0.91     67227

Validation F2 Score:  0.671521477757132
Test Accuracy:  0.903387031996073
Test Confusion Matrix: 
 [[55817  4653]
 [ 1842  4915]]
Test Classification Report: 
               precision    recall  f1-score   support

           0       0.97      0.92      0.95     60470
           1       0.51      0.73      0.60      6757

    accuracy                           0.90     67227
   macro avg       0.74      0.83      0.77     67227
weighted avg       0.92      0.90      0.91     67227

Test F2 Score:  0.671521477757132


# Logistic regression

In [74]:
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, fbeta_score

# Assuming 'pivot' is your final DataFrame and 'is_dropout' is your target variable

# Split data into features (X) and target (y)
X = pivot.drop('is_dropout', axis=1)
y = pivot['is_dropout']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=45)

# Apply standard scaling
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

# Apply SMOTE on the training data
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train_scaled, y_train)

# Now you can fit the model on the training data with L1 regularization
model = LogisticRegression(penalty='l2', random_state=45)
model.fit(X_train_res, y_train_res)

# Predict on the validation set
y_val_pred = model.predict(X_test_scaled)

# Evaluate the model on the validation set
accuracy_val = accuracy_score(y_test, y_val_pred)
confusion_val = confusion_matrix(y_test, y_val_pred)
report_val = classification_report(y_test, y_val_pred)

# Calculate the F2 score on the validation set
f2_val = fbeta_score(y_test, y_val_pred, beta=2)

# Print the evaluation metrics for the validation set
print("Validation Accuracy: ", accuracy_val)
print("Validation Confusion Matrix: \n", confusion_val)
print("Validation Classification Report: \n", report_val)
print("Validation F2 Score: ", f2_val)

# Predict on the test set
y_test_pred = model.predict(X_test_scaled)

# Evaluate the model on the test set
accuracy_test = accuracy_score(y_test, y_test_pred)
confusion_test = confusion_matrix(y_test, y_test_pred)
report_test = classification_report(y_test, y_test_pred)

# Calculate the F2 score on the test set
f2_test = fbeta_score(y_test, y_test_pred, beta=2)

# Print the evaluation metrics for the test set
print("Test Accuracy: ", accuracy_test)
print("Test Confusion Matrix: \n", confusion_test)
print("Test Classification Report: \n", report_test)
print("Test F2 Score: ", f2_test)




Validation Accuracy:  0.8314218989394142
Validation Confusion Matrix: 
 [[50553  9917]
 [ 1416  5341]]
Validation Classification Report: 
               precision    recall  f1-score   support

           0       0.97      0.84      0.90     60470
           1       0.35      0.79      0.49      6757

    accuracy                           0.83     67227
   macro avg       0.66      0.81      0.69     67227
weighted avg       0.91      0.83      0.86     67227

Validation F2 Score:  0.6315328950480065
Test Accuracy:  0.8314218989394142
Test Confusion Matrix: 
 [[50553  9917]
 [ 1416  5341]]
Test Classification Report: 
               precision    recall  f1-score   support

           0       0.97      0.84      0.90     60470
           1       0.35      0.79      0.49      6757

    accuracy                           0.83     67227
   macro avg       0.66      0.81      0.69     67227
weighted avg       0.91      0.83      0.86     67227

Test F2 Score:  0.6315328950480065


lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


# Neural Network

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Assuming 'data' is your input DataFrame with the target column 'is_dropout'

# Identify the categorical columns based on their data types
categorical_cols = data.select_dtypes(include='object').columns.tolist()

# Identify the numeric columns based on their data types
numeric_cols = data.select_dtypes(exclude='object').columns.tolist()

# Split data into features (X) and target (y)
X = data.drop('is_dropout', axis=1)
y = data['is_dropout']

# Perform one-hot encoding on the categorical columns
X_encoded = pd.get_dummies(X, columns=categorical_cols)

# Apply MinMaxScaler on the numeric columns
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_encoded[numeric_cols])

# Combine the scaled numeric features with the encoded categorical features
X_final = pd.concat([pd.DataFrame(X_scaled, columns=numeric_cols), X_encoded.drop(numeric_cols, axis=1)], axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.2, random_state=45)

# Apply SMOTE on the training data
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Build the neural network model
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_res, y_train_res, epochs=10, batch_size=32, verbose=1)

# Predict on the test set
y_pred = model.predict_classes(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Print the evaluation metrics
print("Accuracy: ", accuracy)
print("Confusion Matrix: \n", confusion)
print("Classification Report: \n", report)


# SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, fbeta_score
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler

# Assuming 'pivot' is your final DataFrame and 'is_dropout' is your target variable

# Split data into features (X) and target (y)
X = pivot.drop('is_dropout', axis=1)
y = pivot['is_dropout']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=45)

# Apply standard scaling
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

# Apply SMOTE on the training data
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train_scaled, y_train)

# Calculate the class imbalance ratio
class_counts = y_train_res.value_counts()
imbalance_ratio = class_counts[0] / class_counts[1]

# Define the class weights
class_weights = {0: 1.0, 1: imbalance_ratio}

# Now you can fit the model on the training data
model = SVC(class_weight=class_weights, random_state=45)
model.fit(X_train_res, y_train_res)

# Predict on the validation set
y_val_pred = model.predict(X_test_scaled)

# Evaluate the model on the validation set
accuracy_val = accuracy_score(y_test, y_val_pred)
confusion_val = confusion_matrix(y_test, y_val_pred)
report_val = classification_report(y_test, y_val_pred)

# Calculate the F2 score on the validation set
f2_val = fbeta_score(y_test, y_val_pred, beta=2)

# Print the evaluation metrics for the validation set
print("Validation Accuracy: ", accuracy_val)
print("Validation Confusion Matrix: \n", confusion_val)
print("Validation Classification Report: \n", report_val)
print("Validation F2 Score: ", f2_val)

# Predict on the test set
y_test_pred = model.predict(X_test_scaled)

# Evaluate the model on the test set
accuracy_test = accuracy_score(y_test, y_test_pred)
confusion_test = confusion_matrix(y_test, y_test_pred)
report_test = classification_report(y_test, y_test_pred)

# Calculate the F2 score on the test set
f2_test = fbeta_score(y_test, y_test_pred, beta=2)

# Print the evaluation metrics for the test set
print("Test Accuracy: ", accuracy_test)
print("Test Confusion Matrix: \n", confusion_test)
print("Test Classification Report: \n", report_test)
print("Test F2 Score: ", f2_test)
